In [6]:
# Demo
import tkinter as tk
from tkinter import ttk
import cv2
import numpy as np
from PIL import Image, ImageTk

class OcclusionHandlingTrackerGUI:
    def __init__(self, root):
        # Initialize window
        self.root = root
        self.root.title("Occlusion Handling Tracker")
        self.root.configure(bg="#101820")
        self.root.geometry("750x700")
        
        # Initialize video capture
        self.cap = cv2.VideoCapture(0)
        
        # Tracking variables
        self.tracking = False
        self.tracking_lost = False
        self.tracker = None
        self.bbox = None
        self.prev_points = None
        self.old_gray = None
        self.mask = None
        self.predicted_bbox = None
        self.occlusion_counter = 0
        
        # Tracker options
        self.tracker_types = {
            "CSRT": cv2.legacy.TrackerCSRT_create,
            "KCF": cv2.legacy.TrackerKCF_create, 
            "MIL": cv2.legacy.TrackerMIL_create
        }
        self.tracker_type = tk.StringVar(value="CSRT")
        
        # Optical flow parameters
        self.lk_params = dict(
            winSize=(15, 15),
            maxLevel=2,
            criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
        )
        
        # Occlusion handling parameters
        self.occlusion_threshold = 0.4  # Minimum percentage of points needed
        self.max_occlusion_frames = 15  # Max frames to predict during occlusion
        
        # Create UI elements
        self.create_ui()
        
        # Start video loop
        self.update()
    
    def create_ui(self):
        # Video display
        self.video_lbl = tk.Label(self.root, bg="black")
        self.video_lbl.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Status label
        self.status_lbl = tk.Label(
            self.root, 
            text="Status: Idle", 
            font=("Arial", 15, "bold"), 
            fg="white", 
            bg="#101820"
        )
        self.status_lbl.pack(pady=5)
        
        # Controls frame
        control_frame = tk.Frame(self.root, bg="#2C3E50")
        control_frame.pack(padx=10, pady=10, fill="x")
        
        # Tracker selection
        tk.Label(
            control_frame, 
            text="Select Tracker:", 
            font=("Arial", 12, "bold"), 
            fg="white", 
            bg="#2C3E50"
        ).grid(row=0, column=0, padx=5, pady=5)
        
        tracker_combo = ttk.Combobox(
            control_frame, 
            textvariable=self.tracker_type, 
            values=list(self.tracker_types.keys()), 
            state="readonly"
        )
        tracker_combo.grid(row=0, column=1, padx=5, pady=5)
        
        # Buttons
        tk.Button(
            control_frame, 
            text="Select ROI", 
            width=12, 
            command=self.select_roi, 
            bg="#3498DB", 
            fg="white", 
            font=("Arial", 10, "bold")
        ).grid(row=0, column=2, padx=5, pady=5)
        
        tk.Button(
            control_frame, 
            text="Start Tracking", 
            width=12, 
            command=self.start_tracking, 
            bg="#2ECC71", 
            fg="white", 
            font=("Arial", 10, "bold")
        ).grid(row=0, column=3, padx=5, pady=5)
        
        tk.Button(
            control_frame, 
            text="Stop Tracking", 
            width=12, 
            command=self.stop_tracking, 
            bg="#E74C3C", 
            fg="white", 
            font=("Arial", 10, "bold")
        ).grid(row=0, column=4, padx=5, pady=5)
        
        # Advanced parameters frame
        adv_frame = tk.Frame(self.root, bg="#2C3E50")
        adv_frame.pack(padx=10, pady=5, fill="x")
        
        # Occlusion threshold slider
        tk.Label(
            adv_frame, 
            text="Occlusion Threshold:", 
            font=("Arial", 10),
            fg="white", 
            bg="#2C3E50"
        ).grid(row=0, column=0, padx=5, pady=5)
        
        self.occ_threshold_var = tk.DoubleVar(value=self.occlusion_threshold)
        occ_threshold_slider = ttk.Scale(
            adv_frame, 
            from_=0.1, 
            to=0.9,
            variable=self.occ_threshold_var, 
            orient="horizontal", 
            length=150
        )
        occ_threshold_slider.grid(row=0, column=1, padx=5, pady=5)
        occ_threshold_slider.bind("<ButtonRelease-1>", self.update_parameters)
        
        tk.Label(
            adv_frame, 
            textvariable=self.occ_threshold_var, 
            font=("Arial", 10), 
            fg="white", 
            bg="#2C3E50", 
            width=4
        ).grid(row=0, column=2, padx=5, pady=5)
        
        # Max occlusion frames slider
        tk.Label(
            adv_frame, 
            text="Max Occlusion Frames:", 
            font=("Arial", 10), 
            fg="white", 
            bg="#2C3E50"
        ).grid(row=0, column=3, padx=5, pady=5)
        
        self.max_occ_frames_var = tk.IntVar(value=self.max_occlusion_frames)
        max_occ_frames_slider = ttk.Scale(
            adv_frame, 
            from_=5, 
            to=30, 
            variable=self.max_occ_frames_var, 
            orient="horizontal", 
            length=150
        )
        max_occ_frames_slider.grid(row=0, column=4, padx=5, pady=5)
        max_occ_frames_slider.bind("<ButtonRelease-1>", self.update_parameters)
        
        tk.Label(
            adv_frame, 
            textvariable=self.max_occ_frames_var, 
            font=("Arial", 10), 
            fg="white", 
            bg="#2C3E50", 
            width=4
        ).grid(row=0, column=5, padx=5, pady=5)
    
    def update_parameters(self, event=None):
        # Update parameters when sliders change
        self.occlusion_threshold = self.occ_threshold_var.get()
        self.max_occlusion_frames = self.max_occ_frames_var.get()
    
    def select_roi(self):
        # Pause tracking to select ROI
        self.stop_tracking()
        
        # Get frame for ROI selection
        ret, frame = self.cap.read()
        if ret:
            # Display instruction on frame
            cv2.putText(
                frame, 
                "Select ROI & press ENTER or SPACE", 
                (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, 
                (255, 255, 255), 
                2
            )
            
            # Get ROI from user
            self.bbox = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
            cv2.destroyWindow("Select ROI")
            
            # Initialize tracking if valid ROI
            if self.bbox != (0, 0, 0, 0):
                # Initialize tracker based on selected type
                self.tracker = self.tracker_types[self.tracker_type.get()]()
                self.tracker.init(frame, self.bbox)
                
                # Initialize optical flow
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                self.old_gray = frame_gray
                self.mask = np.zeros_like(frame)
                
                # Create ROI mask for feature detection
                x, y, w, h = [int(v) for v in self.bbox]
                roi_mask = np.zeros_like(frame_gray)
                roi_mask[y:y+h, x:x+w] = 255
                
                # Detect good features in ROI
                self.prev_points = cv2.goodFeaturesToTrack(
                    frame_gray, 
                    mask=roi_mask, 
                    maxCorners=200, 
                    qualityLevel=0.01, 
                    minDistance=7, 
                    blockSize=7
                )
                
                # Set initial prediction
                self.predicted_bbox = (x, y, w, h)
                self.tracking_lost = False
                self.occlusion_counter = 0
                
                # Update status
                self.status_lbl.config(text="Status: ROI Selected", fg="lime")
    
    def start_tracking(self):
        if self.bbox is not None:
            self.tracking = True
            self.status_lbl.config(text="Status: Tracking", fg="lime")
    
    def stop_tracking(self):
        self.tracking = False
        self.status_lbl.config(text="Status: Stopped", fg="orange")
    
    def update(self):
        ret, frame = self.cap.read()
        if ret:
            frame_displayed = frame.copy()
            
            if self.tracking and self.bbox is not None:
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                if not self.tracking_lost:
                    # Update CSRT tracker
                    success, bbox = self.tracker.update(frame)
                    
                    if success:
                        # Extract bbox coordinates
                        x, y, w, h = [int(v) for v in bbox]
                        
                        # Draw bbox
                        cv2.rectangle(frame_displayed, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        cv2.putText(
                            frame_displayed, 
                            "Tracking", 
                            (x, y - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            0.7, 
                            (0, 255, 0), 
                            2
                        )
                        
                        # Update feature points when tracking is good
                        roi_mask = np.zeros_like(self.old_gray)
                        roi_mask[y:y+h, x:x+w] = 255
                        self.prev_points = cv2.goodFeaturesToTrack(
                            self.old_gray, 
                            mask=roi_mask, 
                            maxCorners=200, 
                            qualityLevel=0.01, 
                            minDistance=7, 
                            blockSize=7
                        )
                        self.predicted_bbox = (x, y, w, h)
                        self.occlusion_counter = 0
                        self.status_lbl.config(text="Status: Tracking", fg="lime")
                    else:
                        self.tracking_lost = True
                        self.status_lbl.config(text="Status: Occlusion Detected", fg="#FFA500")
                
                # Use optical flow when tracking is lost or occluded
                if self.tracking_lost and self.prev_points is not None and len(self.prev_points) > 0:
                    # Calculate optical flow
                    new_points, status, err = cv2.calcOpticalFlowPyrLK(
                        self.old_gray, 
                        frame_gray, 
                        self.prev_points, 
                        None, 
                        **self.lk_params
                    )
                    
                    if new_points is not None:
                        # Filter valid points
                        good_new = new_points[status == 1]
                        good_old = self.prev_points[status == 1]
                        
                        # Check if enough points remain
                        if len(good_new) > self.occlusion_threshold * len(self.prev_points):
                            # Calculate median displacement
                            displacement = np.median(good_new - good_old, axis=0)
                            
                            # Update predicted position
                            px, py, pw, ph = self.predicted_bbox
                            new_x = int(px + displacement[0])
                            new_y = int(py + displacement[1])
                            self.predicted_bbox = (new_x, new_y, pw, ph)
                            
                            # Draw predicted bbox
                            x, y, w, h = self.predicted_bbox
                            cv2.rectangle(frame_displayed, (x, y), (x + w, y + h), (0, 255, 255), 2)
                            cv2.putText(
                                frame_displayed, 
                                "Occlusion Handling", 
                                (x, y - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                0.7, 
                                (0, 255, 255), 
                                2
                            )
                            
                            # Update tracker position
                            self.tracker.init(frame, self.predicted_bbox)
                            self.occlusion_counter += 1
                            
                            # Reset if occlusion lasts too long
                            if self.occlusion_counter > self.max_occlusion_frames:
                                self.tracking_lost = True
                                cv2.putText(
                                    frame_displayed, 
                                    "RE-INITIALIZE TRACKER!", 
                                    (50, 50), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, 
                                    (0, 0, 255), 
                                    2
                                )
                                self.status_lbl.config(text="Status: Track Lost - Reselect ROI", fg="red")
                        else:
                            self.tracking_lost = True
                            self.status_lbl.config(text="Status: Track Lost - Reselect ROI", fg="red")
                        
                        # Draw optical flow vectors
                        self.mask = np.zeros_like(frame)
                        for i, (new, old) in enumerate(zip(good_new, good_old)):
                            a, b = new.ravel()
                            c, d = old.ravel()
                            self.mask = cv2.line(
                                self.mask, 
                                (int(a), int(b)), 
                                (int(c), int(d)), 
                                (0, 255, 0), 
                                2
                            )
                            frame_displayed = cv2.circle(
                                frame_displayed, 
                                (int(a), int(b)), 
                                3, 
                                (0, 255, 0), 
                                -1
                            )
                        
                        # Save good points for next iteration
                        self.prev_points = good_new.reshape(-1, 1, 2)
                
                # Reset tracking if completely lost or not enough points
                if (self.tracking_lost and self.occlusion_counter > self.max_occlusion_frames) or \
                   (self.prev_points is not None and len(self.prev_points) < 10):
                    # Prompt users to reselect ROI
                    cv2.putText(
                        frame_displayed, 
                        "Track Lost - Click 'Select ROI'", 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1, 
                        (0, 0, 255), 
                        2
                    )
                    self.status_lbl.config(text="Status: Track Lost - Reselect ROI", fg="red")
                
                # Update previous frame
                self.old_gray = frame_gray.copy()
                
                # Combine optical flow visualization
                frame_displayed = cv2.add(frame_displayed, self.mask)
            
            # Convert to ImageTk format and display
            img = cv2.cvtColor(frame_displayed, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_lbl.imgtk = imgtk
            self.video_lbl.configure(image=imgtk)
        
        # Schedule next update
        self.root.after(15, self.update)
    
    def cleanup(self):
        # Release resources
        if self.cap.isOpened():
            self.cap.release()

# Main function
def main():
    root = tk.Tk()
    app = OcclusionHandlingTrackerGUI(root)
    
    # Set cleanup on window close
    root.protocol("WM_DELETE_WINDOW", lambda: [app.cleanup(), root.destroy()])
    
    # Start main loop
    root.mainloop()

if __name__ == "__main__":
    main()

In [7]:
# CSRT Tracker & Optical Flow & Occlusion handling
import cv2
import numpy as np

# Select object to track (first frame)
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
bbox = cv2.selectROI("Select Object", frame)
cv2.destroyWindow("Select Object")

# Initialize CSRT tracker
tracker = cv2.legacy.TrackerCSRT_create()
tracker.init(frame, bbox)

# Optical flow parameters
lk_params = dict(winSize=(15, 15),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Iocclusion handling
occlusion_threshold = 0.4  # Minimum percentage of points needed to consider tracking valid
occlusion_counter = 0
max_occlusion_frames = 15  # Max frames to predict during occlusion

# Initialize feature points for optical flow
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(frame)
prev_points = cv2.goodFeaturesToTrack(old_gray, maxCorners=200, qualityLevel=0.01, minDistance=7, blockSize=7)

# Convert bbox to integers
x, y, w, h = [int(v) for v in bbox]

# Create ROI mask for feature point selection
roi_mask = np.zeros_like(old_gray)
roi_mask[y:y+h, x:x+w] = 255
prev_points = cv2.goodFeaturesToTrack(old_gray, mask=roi_mask, maxCorners=200, qualityLevel=0.01, minDistance=7, blockSize=7)

# Initialize predicted position
predicted_bbox = (x, y, w, h)
tracking_lost = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if not tracking_lost:
        # Update CSRT tracker
        success, bbox = tracker.update(frame)
        
        if success:
            # Update feature points when tracking is good
            x, y, w, h = [int(v) for v in bbox]
            roi_mask = np.zeros_like(old_gray)
            roi_mask[y:y+h, x:x+w] = 255
            prev_points = cv2.goodFeaturesToTrack(old_gray, mask=roi_mask, maxCorners=200, qualityLevel=0.01, minDistance=7, blockSize=7)
            predicted_bbox = (x, y, w, h)
            occlusion_counter = 0
        else:
            tracking_lost = True

    # Use optical flow when tracking is lost or occluded
    if tracking_lost or (prev_points is not None and len(prev_points) > 0):
        # Calculate optical flow
        new_points, status, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, prev_points, None, **lk_params)
        if new_points is not None:
            # Filter valid points
            good_new = new_points[status == 1]
            good_old = prev_points[status == 1]
            
            # Check if enough points remain
            if len(good_new) > occlusion_threshold * len(prev_points):
                displacement = np.median(good_new - good_old, axis=0) # Calculate median displacement
                
                # Update predicted position
                px, py, pw, ph = predicted_bbox
                new_x = int(px + displacement[0])
                new_y = int(py + displacement[1])
                predicted_bbox = (new_x, new_y, pw, ph)
                
                # Draw predicted bbox
                x, y, w, h = predicted_bbox
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
                cv2.putText(frame, "Occlusion Handling", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                
                # Update tracker position
                tracker.init(frame, predicted_bbox)
                occlusion_counter += 1
                
                # Reset if occlusion lasts too long
                if occlusion_counter > max_occlusion_frames:
                    tracking_lost = True
                    cv2.putText(frame, "RE-INITIALIZE TRACKER!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                tracking_lost = True

            # Draw optical flow vectors
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
                frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 255, 0), -1)
                
            prev_points = good_new.reshape(-1, 1, 2)
            
    # Reset tracking if completely lost
    if tracking_lost:
        bbox = cv2.selectROI("Select Object", frame, False)
        cv2.destroyWindow("Select Object")
        tracker = cv2.legacy.TrackerCSRT_create()
        tracker.init(frame, bbox)
        tracking_lost = False
        prev_points = None

    # Update previous frame and points
    old_gray = frame_gray.copy()
    if prev_points is not None and len(prev_points) < 10:
        tracking_lost = True

    # Combine optical flow visualization
    img = cv2.add(frame, mask)

    cv2.imshow("Occlusion Handling Tracker", img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\dxt.cpp:3506: error: (-215:Assertion failed) type == CV_32FC1 || type == CV_32FC2 || type == CV_64FC1 || type == CV_64FC2 in function 'cv::dft'


In [ ]:
# Main Logic:
# CSRT Tracker & Optical Flow & Kalman Filter  & Tracking History

import cv2
import numpy as np
from collections import deque

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
bbox = cv2.selectROI("Select Object", frame, False)

tracker = cv2.TrackerCSRT_create()
tracker.init(frame, bbox)
prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],[0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],[0, 1, 0, 1],[0, 0, 1, 0],[0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03

tracking_history = deque(maxlen=10)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    success, bbox = tracker.update(frame)

    if success:
        x, y, w, h = [int(v) for v in bbox]
        center_x, center_y = x + w // 2, y + h // 2

        measurement = np.array([[np.float32(center_x)], [np.float32(center_y)]])
        kalman.correct(measurement)

        tracking_history.append((center_x, center_y, w, h))

    else:
        if len(tracking_history) > 1:
            p0 = np.array([[tracking_history[-2][:2]]], dtype=np.float32)
            p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, p0, None)

            if p1 is not None and st[0] == 1:
                pred_x, pred_y = int(p1[0][0][0]), int(p1[0][0][1])
            else:
                prediction = kalman.predict()
                pred_x, pred_y = int(prediction[0]), int(prediction[1])

            bbox = (pred_x - w // 2, pred_y - h // 2, w, h)
            tracker = cv2.TrackerCSRT_create()
            tracker.init(frame, bbox)

    prev_gray = gray.copy()

    if len(tracking_history) > 0:
        last_x, last_y, last_w, last_h = tracking_history[-1]
        cv2.rectangle(frame, (last_x - last_w // 2, last_y - last_h // 2), (last_x + last_w // 2, last_y + last_h // 2), (0, 255, 0), 2)

    for i in range(1, len(tracking_history)):
        cv2.line(frame, tracking_history[i - 1][:2], tracking_history[i][:2], (0, 0, 255), 2)

    cv2.imshow("Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

try OpticalFlow & recovery

In [70]:
import tkinter as tk
from tkinter import ttk
import cv2
import numpy as np
from PIL import Image, ImageTk

# Setup window
win = tk.Tk()
win.title("Object Tracking")
win.configure(bg="#101820")
win.geometry("1100x600")

# Tracker options
types = {"MIL": cv2.TrackerMIL_create, "KCF": cv2.TrackerKCF_create, "CSRT": cv2.TrackerCSRT_create}
tracker_type = tk.StringVar(value="CSRT")

# Tracking enhancement options
use_optical_flow = tk.BooleanVar(value=True)
recovery_mode = tk.BooleanVar(value=True)

# Video capture
cap = cv2.VideoCapture(0)
tracker, bbox, tracking = None, None, False

# Optical flow parameters
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2, 
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Tracking variables
prev_frame = None
prev_pts = None
tracking_history = []  # Store recent tracking positions
recovery_frames = 0    # Count frames since tracking was lost
max_recovery_frames = 30  # Maximum frames to attempt recovery

# UI Elements
video_lbl = tk.Label(win, bg="black")
video_lbl.pack(fill="both", expand=True, padx=10, pady=10)
status_lbl = tk.Label(win, text="Idle", font=("Arial", 15, "bold"), fg="white", bg="#101820")
status_lbl.pack(pady=5)

def select_roi():
    global bbox, tracker, tracking, prev_frame, prev_pts
    ret, frame = cap.read()
    if ret:
        cv2.putText(frame, "Select ROI & press ESC", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        bbox = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
        cv2.destroyWindow("Select ROI")
        if bbox != (0, 0, 0, 0):
            tracker = types[tracker_type.get()]()
            tracker.init(frame, bbox)
            tracking = True
            status_lbl.config(text="Status: Tracking", fg="lime")
            
            # Initialize optical flow
            prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            x, y, w, h = map(int, bbox)
            roi = prev_frame[y:y+h, x:x+w]
            prev_pts = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
            if prev_pts is not None:
                prev_pts = prev_pts + np.array([x, y], dtype=np.float32)  # Adjust coordinates to global frame
            tracking_history.clear()

def start():
    global tracking
    tracking = True
    status_lbl.config(text="Status: Tracking", fg="lime")

def stop():
    global tracking
    tracking = False
    status_lbl.config(text="Status: Stopped", fg="red")

def reset_tracker():
    global tracker, bbox, tracking, prev_frame, prev_pts, tracking_history, recovery_frames
    tracking = False
    tracker = None
    bbox = None
    prev_frame = None
    prev_pts = None
    tracking_history.clear()
    recovery_frames = 0
    status_lbl.config(text="Status: Reset", fg="orange")

def update_tracker_with_optical_flow(frame, gray_frame):
    global prev_frame, prev_pts, bbox
    
    if prev_frame is None or prev_pts is None or len(prev_pts) < 5:
        return False, bbox
    
    # Calculate optical flow
    next_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_frame, gray_frame, prev_pts, None, **lk_params)
    
    # Filter good points
    if next_pts is not None:
        good_new = next_pts[status == 1]
        good_old = prev_pts[status == 1]
        
        if len(good_new) > 5:  # Enough points to estimate movement
            # Calculate bounding box movement
            movement = np.mean(good_new - good_old, axis=0)
            x, y, w, h = bbox
            
            # Update bounding box position
            new_x = x + movement[0]
            new_y = y + movement[1]
            
            # Update previous points for next iteration
            prev_pts = good_new.reshape(-1, 1, 2)
            prev_frame = gray_frame.copy()
            
            # Update bbox
            bbox = (new_x, new_y, w, h)
            return True, bbox
    
    # Not enough points to track
    return False, bbox

def attempt_recovery(frame, gray_frame):
    global bbox, tracker, tracking_history, recovery_frames
    
    if len(tracking_history) < 5:
        return False, bbox
    
    # Predict position based on recent movement
    recent_positions = np.array(tracking_history[-5:])
    if len(recent_positions) >= 2:
        # Calculate average movement direction and speed
        movements = recent_positions[1:] - recent_positions[:-1]
        avg_movement = np.mean(movements, axis=0)
        
        # Predict new position
        last_pos = recent_positions[-1]
        predicted_pos = last_pos + avg_movement * (recovery_frames * 0.5 + 1)  # Scale movement by recovery time
        
        x, y, w, h = bbox
        new_bbox = (predicted_pos[0], predicted_pos[1], w, h)
        
        # Reinitialize tracker at predicted position
        tracker = types[tracker_type.get()]()
        tracker.init(frame, new_bbox)
        
        recovery_frames += 1
        return True, new_bbox
    
    return False, bbox

def update():
    global tracking, tracker, bbox, prev_frame, prev_pts, tracking_history, recovery_frames
    
    ret, frame = cap.read()
    if not ret:
        win.after(15, update)
        return
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if tracking and tracker:
        tracking_success = False
        of_success = False
        
        # 1. Try CSRT tracker first
        if tracker:
            tracking_success, new_bbox = tracker.update(frame)
        
        # 2. Apply optical flow if enabled
        if use_optical_flow.get():
            of_success, of_bbox = update_tracker_with_optical_flow(frame, gray_frame)
            
            # Decide which result to use
            if tracking_success and of_success:
                # Blend CSRT and optical flow results (weighted average)
                csrt_weight = 0.7
                of_weight = 0.3
                x1, y1, w1, h1 = new_bbox
                x2, y2, w2, h2 = of_bbox
                
                blended_x = x1 * csrt_weight + x2 * of_weight
                blended_y = y1 * csrt_weight + y2 * of_weight
                bbox = (blended_x, blended_y, w1, h1)  # Keep original width/height
                
                # Update optical flow points occasionally for long-term stability
                if len(tracking_history) % 10 == 0:
                    x, y, w, h = map(int, bbox)
                    roi = gray_frame[max(0, y):min(frame.shape[0], y+h), 
                                    max(0, x):min(frame.shape[1], x+w)]
                    if roi.size > 0:
                        pts = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
                        if pts is not None and pts.size > 0:
                            prev_pts = pts + np.array([x, y], dtype=np.float32)
                            prev_frame = gray_frame.copy()
            elif tracking_success:
                bbox = new_bbox
            elif of_success:
                bbox = of_bbox
                
        elif tracking_success:
            bbox = new_bbox
        
        # Store tracking history (for recovery)
        if tracking_success or of_success:
            x, y, w, h = map(float, bbox)
            center_x, center_y = x + w/2, y + h/2
            tracking_history.append((center_x, center_y))
            if len(tracking_history) > 30:  # Keep last 30 positions
                tracking_history.pop(0)
            
            recovery_frames = 0
            status_lbl.config(text="Status: Tracking", fg="lime")
        
        # Attempt recovery if tracking lost
        elif recovery_mode.get() and recovery_frames < max_recovery_frames:
            recovery_success, recovery_bbox = attempt_recovery(frame, gray_frame)
            if recovery_success:
                bbox = recovery_bbox
                status_lbl.config(text=f"Status: Recovering ({recovery_frames}/{max_recovery_frames})", fg="orange")
            else:
                status_lbl.config(text="Status: Lost Tracking", fg="red")
        else:
            status_lbl.config(text="Status: Lost Tracking", fg="red")
            
        # Draw tracking information on frame
        if bbox is not None:
            x, y, w, h = map(int, bbox)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            if tracking_success:
                label = "Tracking (CSRT)"
            elif of_success:
                label = "Tracking (OptFlow)"
            elif recovery_frames > 0:
                label = f"Recovering ({recovery_frames})"
            else:
                label = "Lost"
                
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Draw tracking history
            if len(tracking_history) > 1:
                for i in range(1, len(tracking_history)):
                    pt1 = tuple(map(int, tracking_history[i-1]))
                    pt2 = tuple(map(int, tracking_history[i]))
                    cv2.line(frame, pt1, pt2, (0, 0, 255), 1)
    
    # Display frame
    img = ImageTk.PhotoImage(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
    video_lbl.imgtk = img
    video_lbl.configure(image=img)
    
    win.after(33, update)

# Controls
frame = tk.Frame(win, bg="#2C3E50")
frame.pack(padx=10, pady=10)
tk.Label(frame, text="Select Tracker:", font=("Arial", 12, "bold"), fg="white", bg="#2C3E50").grid(row=0, column=0, padx=5, pady=5)
tt = ttk.Combobox(frame, textvariable=tracker_type, values=list(types.keys()), state="readonly").grid(row=0, column=1, padx=5, pady=5)
tk.Button(frame, text="Select ROI", width=12, command=select_roi, bg="#3498DB", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=2, padx=5, pady=5)
tk.Button(frame, text="Start Tracking", width=12, command=start, bg="#2ECC71", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=3, padx=5, pady=5)
tk.Button(frame, text="Stop Tracking", width=12, command=stop, bg="#E74C3C", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=4, padx=5, pady=5)
tk.Button(frame, text="Reset", width=12, command=reset_tracker, bg="#F39C12", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=5, padx=5, pady=5)
tk.Checkbutton(frame, text="Use Optical Flow", variable=use_optical_flow, fg="white", bg="#2C3E50", selectcolor="#3498DB", font=("Arial", 10)).grid(row=0, column=6, padx=15, pady=5)
tk.Checkbutton(frame, text="Auto Recovery Mode", variable=recovery_mode,fg="white", bg="#2C3E50", selectcolor="#3498DB", font=("Arial", 10)).grid(row=0, column=7, padx=15, pady=5)

# Main loop
update()
win.mainloop()
cap.release()
cv2.destroyAllWindows()

try kalman filtering 

In [65]:
import tkinter as tk
from tkinter import ttk
import cv2
import numpy as np
from PIL import Image, ImageTk

# Setup window
win = tk.Tk()
win.title("Object Tracking with Kalman Filter")
win.configure(bg="#101820")
win.geometry("1000x650")

# Tracker options
types = {
    "MIL": cv2.TrackerMIL_create, 
    "KCF": cv2.TrackerKCF_create, 
    "CSRT": cv2.TrackerCSRT_create
}
tracker_type = tk.StringVar(value="CSRT")

# Video capture
cap = cv2.VideoCapture(0)
tracker, bbox, tracking = None, None, False
use_kalman = tk.BooleanVar(value=True)

# Kalman Filter setup
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 0.03
kalman_initialized = False

# UI Elements
video_lbl = tk.Label(win, bg="black")
video_lbl.pack(fill="both", expand=True)
status_lbl = tk.Label(win, text="Idle", font=("Arial", 15, "bold"), fg="white", bg="#101820")
status_lbl.pack()



def select_roi():
    global bbox, tracker, tracking, kalman_initialized
    ret, frame = cap.read()
    if ret:
        cv2.putText(frame, "Select ROI & press ENTER", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        bbox = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
        cv2.destroyWindow("Select ROI")
        if bbox != (0, 0, 0, 0):
            x, y, w, h = bbox
            center_x = x + w/2
            center_y = y + h/2
            
            # Initialize Kalman filter with object center
            kalman.statePre = np.array([[center_x], [center_y], [0], [0]], np.float32)
            kalman.statePost = np.array([[center_x], [center_y], [0], [0]], np.float32)
            kalman_initialized = True
            
            # Initialize tracker
            tracker = types[tracker_type.get()]()
            tracker.init(frame, bbox)
            tracking = True
            status_lbl.config(text="Status: Tracking", fg="lime")

def start():
    global tracking
    if bbox is not None:
        tracking = True
        status_lbl.config(text="Status: Tracking", fg="lime")

def stop():
    global tracking
    tracking = False
    status_lbl.config(text="Status: Stopped", fg="red")

def reset_kalman():
    global kalman_initialized
    kalman_initialized = False

def update():
    global kalman_initialized, bbox
    ret, frame = cap.read()
    if ret:
        if tracking and tracker:
            success, new_bbox = tracker.update(frame)
            
            if success:
                # Get current position
                x, y, w, h = map(int, new_bbox)
                center_x = x + w/2
                center_y = y + h/2
                
                # Draw tracker result in green
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.circle(frame, (int(center_x), int(center_y)), 5, (0, 255, 0), -1)
                cv2.putText(frame, "Tracker", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                # Use Kalman filter if enabled
                if use_kalman.get():
                    if not kalman_initialized:
                        # Initialize Kalman with current position
                        kalman.statePre = np.array([[center_x], [center_y], [0], [0]], np.float32)
                        kalman.statePost = np.array([[center_x], [center_y], [0], [0]], np.float32)
                        kalman_initialized = True
                    
                    # Correct using measurement
                    measurement = np.array([[center_x], [center_y]], np.float32)
                    kalman.correct(measurement)
                    
                    # Predict next position
                    prediction = kalman.predict()
                    pred_x, pred_y = int(prediction[0]), int(prediction[1])
                    
                    # Draw Kalman prediction in blue
                    cv2.circle(frame, (pred_x, pred_y), 5, (255, 0, 0), -1)
                    cv2.putText(frame, "Kalman", (pred_x + 10, pred_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                    
                    # Draw a line connecting actual position to prediction
                    cv2.line(frame, (int(center_x), int(center_y)), (pred_x, pred_y), (255, 255, 0), 1)
                    
                    # Update status with position info
                    status_lbl.config(text=f"Tracking: Actual ({int(center_x)},{int(center_y)}) | Predicted ({pred_x},{pred_y})", fg="lime")
                else:
                    status_lbl.config(text=f"Tracking: Position ({int(center_x)},{int(center_y)})", fg="lime")
            else:
                if use_kalman.get() and kalman_initialized:
                    # If tracking is lost, still predict with Kalman
                    prediction = kalman.predict()
                    pred_x, pred_y = int(prediction[0]), int(prediction[1])
                    
                    # Draw only Kalman prediction in blue
                    cv2.circle(frame, (pred_x, pred_y), 8, (255, 0, 0), -1)
                    cv2.putText(frame, "Kalman (Lost Tracking)", (pred_x + 10, pred_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                    status_lbl.config(text=f"Lost Tracking: Kalman predicting ({pred_x},{pred_y})", fg="orange")
                    
                    # Update the tracker with Kalman's prediction if confidence is lost
                    pred_w, pred_h = (bbox[2], bbox[3]) if bbox else (50, 50)
                    new_bbox = (pred_x - pred_w//2, pred_y - pred_h//2, pred_w, pred_h)
                    tracker.init(frame, new_bbox)
                else:
                    status_lbl.config(text="Status: Lost Tracking", fg="orange")
        
        # Add info text
        cv2.putText(frame, f"Tracker: {tracker_type.get()}", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        cv2.putText(frame, f"Kalman: {'Enabled' if use_kalman.get() else 'Disabled'}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        # Display the frame
        img = ImageTk.PhotoImage(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
        video_lbl.imgtk = img
        video_lbl.configure(image=img)
    
    win.after(15, update)

# Controls frame
frame = tk.Frame(win, bg="#2C3E50")
frame.pack(padx=10, pady=10)

# First row controls
tk.Label(frame, text="Select Tracker:", font=("Arial", 12, "bold"), fg="white", bg="#2C3E50").grid(row=0, column=0)
ttk.Combobox(frame, textvariable=tracker_type, values=list(types.keys()), state="readonly").grid(row=0, column=1, padx=5)
tk.Button(frame, text="Select ROI", width=12, command=select_roi, bg="#3498DB", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=2 , padx=5 , pady=5)
tk.Button(frame, text="Start Tracking", width=12, command=start, bg="#2ECC71", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=3 , padx=5, pady=5)
tk.Button(frame, text="Stop Tracking", width=12, command=stop, bg="#E74C3C", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=4 , padx=5, pady=5)
tk.Button(frame, text="Reset Kalman", width=12, command=reset_kalman, bg="#F39C12", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=5 , padx=5, pady=5)
tk.Checkbutton(frame, text="Use Kalman Filter", variable=use_kalman, onvalue=True, offvalue=False, bg="#2C3E50", fg="white", selectcolor="#2C3E50", font=("Arial", 10, "bold")).grid(row=0, column=6, padx=5)

# Main loop
update()
win.mainloop()
cap.release()
cv2.destroyAllWindows()

C:\Users\gemy2\AppData\Local\Temp\ipykernel_11808\4170823917.py:111: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(prediction[0]), int(prediction[1])


try oop

In [56]:
import tkinter as tk
from tkinter import ttk
import cv2
import numpy as np
from PIL import Image, ImageTk

class ObjectTrackingApp:
    def __init__(self, root):
        # Initialize the main window
        self.root = root
        self.root.title("Enhanced Object Tracking")
        self.root.configure(bg="#101820")
        self.root.geometry("1100x600")
        
        # Tracker options
        self.tracker_types = {
            "MIL": cv2.TrackerMIL_create, 
            "KCF": cv2.TrackerKCF_create, 
            "CSRT": cv2.TrackerCSRT_create
        }
        self.tracker_type = tk.StringVar(value="CSRT")
        
        # Tracking enhancement options
        self.use_optical_flow = tk.BooleanVar(value=True)
        self.recovery_mode = tk.BooleanVar(value=True)
        
        # Tracking variables
        self.cap = cv2.VideoCapture(0)
        self.tracker = None
        self.bbox = None
        self.tracking = False
        self.prev_frame = None
        self.prev_pts = None
        self.tracking_history = []
        self.recovery_frames = 0
        self.max_recovery_frames = 30
        
        # Optical flow parameters
        self.feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
        self.lk_params = dict(winSize=(15, 15), maxLevel=2, 
                         criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
        
        # Create UI Elements
        self._create_ui()
        
    def _create_ui(self):
        # Video display label
        self.video_lbl = tk.Label(self.root, bg="black")
        self.video_lbl.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Status label
        self.status_lbl = tk.Label(self.root, text="Idle", font=("Arial", 15, "bold"), fg="white", bg="#101820")
        self.status_lbl.pack(pady=5)
        
        # Controls frame
        frame = tk.Frame(self.root, bg="#2C3E50")
        frame.pack(padx=10, pady=10)
        
        # Control elements
        tk.Label(frame, text="Select Tracker:", font=("Arial", 12, "bold"), fg="white", bg="#2C3E50").grid(row=0, column=0, padx=5, pady=5)
        ttk.Combobox(frame, textvariable=self.tracker_type, values=list(self.tracker_types.keys()), state="readonly").grid(row=0, column=1, padx=5, pady=5)
        tk.Button(frame, text="Select ROI", width=12, command=self.select_roi, bg="#3498DB", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=2, padx=5, pady=5)
        tk.Button(frame, text="Start Tracking", width=12, command=self.start, bg="#2ECC71", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=3, padx=5, pady=5)
        tk.Button(frame, text="Stop Tracking", width=12, command=self.stop, bg="#E74C3C", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=4, padx=5, pady=5)
        tk.Button(frame, text="Reset", width=12, command=self.reset_tracker, bg="#F39C12", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=5, padx=5, pady=5)
        tk.Checkbutton(frame, text="Use Optical Flow", variable=self.use_optical_flow, fg="white", bg="#2C3E50", selectcolor="#3498DB", font=("Arial", 10)).grid(row=0, column=6, padx=15, pady=5)
        tk.Checkbutton(frame, text="Auto Recovery Mode", variable=self.recovery_mode, fg="white", bg="#2C3E50", selectcolor="#3498DB", font=("Arial", 10)).grid(row=0, column=7, padx=15, pady=5)
    
    def select_roi(self):
        ret, frame = self.cap.read()
        if ret:
            cv2.putText(frame, "Select ROI & press ESC", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            self.bbox = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
            cv2.destroyWindow("Select ROI")
            if self.bbox != (0, 0, 0, 0):
                self.tracker = self.tracker_types[self.tracker_type.get()]()
                self.tracker.init(frame, self.bbox)
                self.tracking = True
                self.status_lbl.config(text="Status: Tracking", fg="lime")
                
                # Initialize optical flow
                self.prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                x, y, w, h = map(int, self.bbox)
                roi = self.prev_frame[y:y+h, x:x+w]
                self.prev_pts = cv2.goodFeaturesToTrack(roi, mask=None, **self.feature_params)
                if self.prev_pts is not None:
                    self.prev_pts = self.prev_pts + np.array([x, y], dtype=np.float32)  # Adjust coordinates to global frame
                self.tracking_history.clear()
    
    def start(self):
        self.tracking = True
        self.status_lbl.config(text="Status: Tracking", fg="lime")
    
    def stop(self):
        self.tracking = False
        self.status_lbl.config(text="Status: Stopped", fg="red")
    
    def reset_tracker(self):
        self.tracking = False
        self.tracker = None
        self.bbox = None
        self.prev_frame = None
        self.prev_pts = None
        self.tracking_history.clear()
        self.recovery_frames = 0
        self.status_lbl.config(text="Status: Reset", fg="orange")
    
    def update_tracker_with_optical_flow(self, frame, gray_frame):
        if self.prev_frame is None or self.prev_pts is None or len(self.prev_pts) < 5:
            return False, self.bbox
        
        # Calculate optical flow
        next_pts, status, _ = cv2.calcOpticalFlowPyrLK(self.prev_frame, gray_frame, self.prev_pts, None, **self.lk_params)
        
        # Filter good points
        if next_pts is not None:
            good_new = next_pts[status == 1]
            good_old = self.prev_pts[status == 1]
            
            if len(good_new) > 5:  # Enough points to estimate movement
                # Calculate bounding box movement
                movement = np.mean(good_new - good_old, axis=0)
                x, y, w, h = self.bbox
                
                # Update bounding box position
                new_x = x + movement[0]
                new_y = y + movement[1]
                
                # Update previous points for next iteration
                self.prev_pts = good_new.reshape(-1, 1, 2)
                self.prev_frame = gray_frame.copy()
                
                # Update bbox
                new_bbox = (new_x, new_y, w, h)
                return True, new_bbox
        
        # Not enough points to track
        return False, self.bbox
    
    def attempt_recovery(self, frame, gray_frame):
        if len(self.tracking_history) < 5:
            return False, self.bbox
        
        # Predict position based on recent movement
        recent_positions = np.array(self.tracking_history[-5:])
        if len(recent_positions) >= 2:
            # Calculate average movement direction and speed
            movements = recent_positions[1:] - recent_positions[:-1]
            avg_movement = np.mean(movements, axis=0)
            
            # Predict new position
            last_pos = recent_positions[-1]
            predicted_pos = last_pos + avg_movement * (self.recovery_frames * 0.5 + 1)  # Scale movement by recovery time
            
            x, y, w, h = self.bbox
            new_bbox = (predicted_pos[0], predicted_pos[1], w, h)
            
            # Reinitialize tracker at predicted position
            self.tracker = self.tracker_types[self.tracker_type.get()]()
            self.tracker.init(frame, new_bbox)
            
            self.recovery_frames += 1
            return True, new_bbox
        
        return False, self.bbox
    
    def update(self):
        ret, frame = self.cap.read()
        if not ret:
            self.root.after(15, self.update)
            return
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        if self.tracking and self.tracker:
            tracking_success = False
            of_success = False
            
            # 1. Try CSRT tracker first
            if self.tracker:
                tracking_success, new_bbox = self.tracker.update(frame)
            
            # 2. Apply optical flow if enabled
            if self.use_optical_flow.get():
                of_success, of_bbox = self.update_tracker_with_optical_flow(frame, gray_frame)
                
                # Decide which result to use
                if tracking_success and of_success:
                    # Blend CSRT and optical flow results (weighted average)
                    csrt_weight = 0.7
                    of_weight = 0.3
                    x1, y1, w1, h1 = new_bbox
                    x2, y2, w2, h2 = of_bbox
                    
                    blended_x = x1 * csrt_weight + x2 * of_weight
                    blended_y = y1 * csrt_weight + y2 * of_weight
                    self.bbox = (blended_x, blended_y, w1, h1)  # Keep original width/height
                    
                    # Update optical flow points occasionally for long-term stability
                    if len(self.tracking_history) % 10 == 0:
                        x, y, w, h = map(int, self.bbox)
                        roi = gray_frame[max(0, y):min(frame.shape[0], y+h), 
                                        max(0, x):min(frame.shape[1], x+w)]
                        if roi.size > 0:
                            pts = cv2.goodFeaturesToTrack(roi, mask=None, **self.feature_params)
                            if pts is not None and pts.size > 0:
                                self.prev_pts = pts + np.array([x, y], dtype=np.float32)
                                self.prev_frame = gray_frame.copy()
                elif tracking_success:
                    self.bbox = new_bbox
                elif of_success:
                    self.bbox = of_bbox
                    
            elif tracking_success:
                self.bbox = new_bbox
            
            # Store tracking history (for recovery)
            if tracking_success or of_success:
                x, y, w, h = map(float, self.bbox)
                center_x, center_y = x + w/2, y + h/2
                self.tracking_history.append((center_x, center_y))
                if len(self.tracking_history) > 30:  # Keep last 30 positions
                    self.tracking_history.pop(0)
                
                self.recovery_frames = 0
                self.status_lbl.config(text="Status: Tracking", fg="lime")
            
            # Attempt recovery if tracking lost
            elif self.recovery_mode.get() and self.recovery_frames < self.max_recovery_frames:
                recovery_success, recovery_bbox = self.attempt_recovery(frame, gray_frame)
                if recovery_success:
                    self.bbox = recovery_bbox
                    self.status_lbl.config(text=f"Status: Recovering ({self.recovery_frames}/{self.max_recovery_frames})", fg="orange")
                else:
                    self.status_lbl.config(text="Status: Lost Tracking", fg="red")
            else:
                self.status_lbl.config(text="Status: Lost Tracking", fg="red")
                
            # Draw tracking information on frame
            if self.bbox is not None:
                x, y, w, h = map(int, self.bbox)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                if tracking_success:
                    label = "Tracking (CSRT)"
                elif of_success:
                    label = "Tracking (OptFlow)"
                elif self.recovery_frames > 0:
                    label = f"Recovering ({self.recovery_frames})"
                else:
                    label = "Lost"
                    
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                # Draw tracking history
                if len(self.tracking_history) > 1:
                    for i in range(1, len(self.tracking_history)):
                        pt1 = tuple(map(int, self.tracking_history[i-1]))
                        pt2 = tuple(map(int, self.tracking_history[i]))
                        cv2.line(frame, pt1, pt2, (0, 0, 255), 1)
        
        # Display frame
        img = ImageTk.PhotoImage(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
        self.video_lbl.imgtk = img
        self.video_lbl.configure(image=img)
        
        self.root.after(33, self.update)
    
    def run(self):
        # Start the update loop and run the application
        self.update()
        self.root.mainloop()
    
    def cleanup(self):
        # Release resources
        self.cap.release()
        cv2.destroyAllWindows()


# Entry point
if __name__ == "__main__":
    root = tk.Tk()
    app = ObjectTrackingApp(root)
    try:
        app.run()
    finally:
        app.cleanup()

#### try!!!!!!

##### Kalman Filter Prediction
Adds motion prediction to handle temporary occlusions
Smooths tracking trajectory to reduce jitter
Provides position estimates when other tracking methods fail

##### Template Matching
Stores an image template of the tracked object, then searches for it when the primary tracker fails, helping re-identify the object after occlusion.

##### Motion Detection
Uses a background subtractor to detect moving objects in the region where the tracked object was last seen.

##### Confidence Scoring
Calculates and displays a confidence value for tracking quality, helping determine when to trust the tracker versus when to use recovery methods.

##### Object Re-acquisition:
Saves templates of the tracked object
Uses template matching to relocate lost objects
Implements a confidence threshold for reliable recovery

##### Multi-method Fusion:
Combines traditional trackers (CSRT, KCF, MIL) with optical flow and Kalman filtering
Displays which tracking method is currently active
Falls back gracefully between methods when occlusions occur